In [1]:
"""
민원이 접수되면 이 민원이 어느 부서에서 처리하는 지 분류하는 것이
실제 업무에서는 매우 중요한 일이다.
그렇기 때문에 모든 민원에는 처리부서가 있어야 한다.

대부분 민원의 경우 처리부서 label이 있지만
몇몇 민원의 경우 처리부서가 공란으로 남겨져 있다.

학습용과 검증용으로 label이 붙어있는 민원 중
20% 정도의 처리부서를 지워 공란이 되었다고 가정하고
모델 학습을 진행한다.

그 후 공란으로 남겨진 데이터에 대하여 label을 붙인다.
"""
print('Hello, World!')

Hello, World!


In [2]:
import random
from time import time
from datetime import datetime

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [3]:
# 데이터 로드
import pandas as pd
from google.colab import auth, drive
auth.authenticate_user()
drive.mount('/content/gdrive')
original_complaints = pd.read_csv('gdrive/My Drive/Colab Notebooks/complaints_data/complaints.csv', encoding='UTF-8')
print(original_complaints)
print(original_complaints.columns)

Mounted at /content/gdrive
                민원접수번호*  ...                                              _명사추출
0      1AA-2007-0000036  ...                                               리플렉터
1      1AA-2007-0001346  ...                     소화전 불법 주정 차 신고 동일 위치 촬영 과태료 부과
2      1AA-2007-0001565  ...  소화전 불법 주정 차 신고 안전 신문고 소방 시설 주정 차 신고 시 적색 주정 차 ...
3      1AA-2007-0001575  ...                        소화전 불법 주정 차 신고 해당 차량 과태료 부과
4      1AA-2007-0001694  ...                         생활 불편 신고 신고 시 과태료 부과 가능 시간
...                 ...  ...                                                ...
21177  1BA-2008-0688825  ...                                   가로수 뿌리 개인 하수관 파손
21178  1BA-2008-1002135  ...                               자동차 수리비 견적 정확 수리 수리비
21179  1BA-2008-1067019  ...                             어린이집 위생 안전 관리 지도 점검 요청
21180  1BA-2009-0053881  ...                                내 아파트 미 착용 공사 소음 불편
21181  1BA-2009-0734245  ...                                        민원 서류 보완 요청

[21182 rows 

In [4]:
# x, y 및 train, test 나누기

words = original_complaints['_명사추출'].fillna('')  # 기준이 될 x값
tfidf_vectorizer = TfidfVectorizer()
x = tfidf_vectorizer.fit_transform(words).toarray()
x = x.reshape((x.shape[0], x.shape[1], 1))

label = original_complaints['처리부서*']  # 기준이 될 y값
one_hot_encoder = OneHotEncoder()
label_encoder = LabelEncoder()
label = label.to_numpy().reshape(-1, 1)
label_num = label_encoder.fit_transform(label).reshape(-1, 1)
y = one_hot_encoder.fit_transform(label_num).toarray()

print('X shape:', x.shape)
print('Y shape:', y.shape)

X shape: (21182, 3450, 1)
Y shape: (21182, 38)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
# row 나누기
# colab 세션 다운으로 인해 데이터 나눔

filled_id = [idx for idx, is_blank in enumerate(original_complaints['처리부서*']=='부서없음') if not is_blank]
blank_id = [idx for idx, is_blank in enumerate(original_complaints['처리부서*']=='부서없음') if is_blank]

complaints_filled = original_complaints.loc[filled_id,]
complaints_blank = original_complaints.loc[blank_id,]

sample_num = 100

sample_id = random.sample(filled_id, sample_num)
train_id = random.sample(sample_id, int(sample_num * 0.8))
test_id = [i for i in sample_id if i not in train_id]

complaints = complaints_filled.loc[sample_id,]

x_train, y_train = x[train_id], y[train_id]
x_test, y_test = x[test_id], y[test_id]
x_blank, y_blank = x[blank_id], x[blank_id]

print('filled :', complaints_filled.shape)
print('blank :', complaints_blank.shape)
print('sample :', complaints.shape)

filled : (20043, 9)
blank : (1139, 9)
sample : (100, 9)


In [8]:
# 모델 생성
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(x.shape[1], 1)))
model.add(LSTM(100, activation='relu', input_shape=(200, 1)))  # 리턴시퀀스는 마지막꺼는 빼고
# DENSE와 사용법 동일하나 input_shape=(열, 몇개씩잘라작업)
# model.add(Dense(5))
model.add(Dense(y.shape[1], activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3450, 200)         161600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense (Dense)                (None, 38)                3838      
Total params: 285,838
Trainable params: 285,838
Non-trainable params: 0
_________________________________________________________________


In [9]:
# 모델 실행
start = datetime.now()
print(f'시작시간: {start}')
adam_opt = keras.optimizers.Adam(lr=0.001, decay=0.9)
model.compile(optimizer=adam_opt, loss='mse')
model.fit(x_train, y_train, epochs=3, batch_size=40)
end = datetime.now()
print(f'종료시간: {end}')
print(f'소요시간: {end - start}')

시작시간: 2021-01-24 13:49:14.386725
Epoch 1/3
2/2 [==============================] - 27s 12s/step - loss: 0.0256
Epoch 2/3
2/2 [==============================] - 24s 12s/step - loss: 0.0256
Epoch 3/3
2/2 [==============================] - 24s 12s/step - loss: 0.0256
종료시간: 2021-01-24 13:50:29.718552
소요시간: 0:01:15.331827


In [11]:
# 결과 확인
y_pred = model.predict(x_test)
y_pred_argmax = np.argmax(y_pred, axis=1)
# complaints.loc[test_id, '분류부서'] = label_encoder.inverse_transform(y_pred_argmax)
for idx, t_id in enumerate(test_id):
    complaints.loc[t_id, '분류부서'] = label_encoder.inverse_transform(y_pred_argmax)[idx]

correct = 0
miss = 0
miss_dict = dict()
for idx in complaints.loc[test_id,].index:
    answer = complaints.loc[idx, '처리부서*']
    predic = complaints.loc[idx, '분류부서']
    if answer == predic:
        correct += 1
    else:
        miss += 1
        miss_dict[f'{answer} {predic}'] = miss_dict.get(f'{answer} {predic}', 0) + 1
print('정분류 :', correct)
print('오분류 :', miss)
print()
for m in miss_dict:
    print(m, miss_dict[m])

정분류 : 13
오분류 : 7
민원봉사과 교통행정과 2
환경보전과 교통행정과 1
자동차관리과 교통행정과 1
공동주택과 교통행정과 1
노인장애인과 교통행정과 1
도시경관과 교통행정과 1


In [14]:
y_pred = model.predict(x_blank)
y_pred_argmax = np.argmax(y_pred, axis=1)
for idx, t_id in enumerate(test_id):
    original_complaints.loc[t_id, '분류부서'] = label_encoder.inverse_transform(y_pred_argmax)[idx]

In [15]:
now = datetime.strftime(datetime.now(), '%m%d_%H%M')
original_complaints.to_csv(f'gdrive/My Drive/Colab Notebooks/complaints_data/complaints_model_{now}.csv', index=False, encoding='UTF-8')
print('저장완료!')

저장완료
